In [ ]:
!pip install transformers datasets torch --quiet


In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments


In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
texts = [
    "This product is amazing and works very well",
    "I am extremely disappointed with the service",
    "The movie was fantastic",
    "The experience was terrible",
    "I really enjoyed using this application",
    "The support team was very unhelpful"
]

labels = [1, 0, 1, 0, 1, 0]  # 1 = Positive, 0 = Negative


In [ ]:
encodings = tokenizer(
    texts,
    truncation=True,
    padding=True,
    return_tensors="pt"
)


In [ ]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

dataset = TextDataset(encodings, labels)


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

trainer.train()


Step,Training Loss


TrainOutput(global_step=6, training_loss=0.6885704199473063, metrics={'train_runtime': 8.0824, 'train_samples_per_second': 1.485, 'train_steps_per_second': 0.742, 'total_flos': 34151751216.0, 'train_loss': 0.6885704199473063, 'epoch': 2.0})

In [ ]:
test_text = "The service was very good and satisfying"

inputs = tokenizer(
    test_text,
    return_tensors="pt",
    truncation=True,
    padding=True
)

# Move inputs to the same device as the model
inputs = {key: value.to(model.device) for key, value in inputs.items()}

outputs = model(**inputs)
prediction = torch.argmax(outputs.logits, dim=1)

if prediction.item() == 1:
    print("Positive Sentiment")
else:
    print("Negative Sentiment")

Positive Sentiment


In [ ]:
model.save_pretrained("distilbert_model")
tokenizer.save_pretrained("distilbert_model")


('distilbert_model/tokenizer_config.json',
 'distilbert_model/special_tokens_map.json',
 'distilbert_model/vocab.txt',
 'distilbert_model/added_tokens.json')